# **1. 데이터 확인**

In [2]:
import pandas as pd
from pathlib import Path
import re

In [3]:
csv_test = pd.read_csv('C:/ESG_Project1/file/solar_data_file/2022/남동발전량_2022_01.csv',
                       encoding='utf-8-sig')

csv_test.head(5)

,발전구분,호기,일자,1시 발전량(MWh),2시 발전량(MWh),3시 발전량(MWh),4시 발전량(MWh),5시 발전량(MWh),6시 발전량(MWh),7시 발전량(MWh),...,21시 발전량(MWh),22시 발전량(MWh),23시 발전량(MWh),24시 발전량(MWh),총량(KW),평균(KW),최대(시간별),최소(시간별),최대,최소
영동태양광,1,2022-01-31 00:00:00,0.0,0,0,0,0,0,0,0.000,...,590.688,590.4,128.448,4583.424,191,593.76,37.44,593.760,0,
삼천포태양광#5,1,2022-01-31 00:00:00,0.0,0,0,0,0,0,0,3.820,...,922.580,0.0,0.000,19116.254,797,2523.68,3.82,2523.683,0,
삼천포태양광#5,2,2022-01-31 00:00:00,0.0,0,0,0,0,0,0,5.094,...,1260.214,0.0,0.000,34126.764,1422,4692.76,5.09,4692.769,0,
광양항세방태양광,1,2022-01-31 00:00:00,0.0,0,0,0,0,0,0,4.800,...,0.000,0.0,0.000,9499.200,396,1627.20,4.80,1627.200,0,
두산엔진MG태양광,1,2022-01-31 00:00:00,0.0,0,0,0,0,0,0,0.000,...,0.000,0.0,0.000,275.460,11,50.04,1.44,50.040,0,


# **2. 수집한 데이터의 컬럼(열) 구성 확인**

In [10]:
print(csv_test.columns.tolist())
print(csv_test.iloc[0].to_dict())

['발전구분', ' 호기', ' 일자', ' 1시 발전량(MWh)', ' 2시 발전량(MWh)', '3시 발전량(MWh)', '4시 발전량(MWh)', '5시 발전량(MWh)', '6시 발전량(MWh)', '7시 발전량(MWh)', '8시 발전량(MWh)', '9시 발전량(MWh)', '10시 발전량(MWh)', '11시 발전량(MWh)', '12시 발전량(MWh)', '13시 발전량(MWh)', '14시 발전량(MWh)', '15시 발전량(MWh)', '16시 발전량(MWh)', '17시 발전량(MWh)', '18시 발전량(MWh)', '19시 발전량(MWh)', '20시 발전량(MWh)', '21시 발전량(MWh)', '22시 발전량(MWh)', '23시 발전량(MWh)', '24시 발전량(MWh)', '총량(KW)', '평균(KW)', '최대(시간별)', '최소(시간별)', '최대', '최소']
{'발전구분': 1, ' 호기': ' 2022-01-31 00:00:00', ' 일자': 0.0, ' 1시 발전량(MWh)': 0, ' 2시 발전량(MWh)': 0, '3시 발전량(MWh)': 0, '4시 발전량(MWh)': 0, '5시 발전량(MWh)': 0, '6시 발전량(MWh)': 0, '7시 발전량(MWh)': 0.0, '8시 발전량(MWh)': 60.192, '9시 발전량(MWh)': 304.032, '10시 발전량(MWh)': 167.616, '11시 발전량(MWh)': 37.44, '12시 발전량(MWh)': 38.496, '13시 발전량(MWh)': 40.896, '14시 발전량(MWh)': 593.76, '15시 발전량(MWh)': 553.728, '16시 발전량(MWh)': 377.76, '17시 발전량(MWh)': 112.416, '18시 발전량(MWh)': 201.696, '19시 발전량(MWh)': 291.936, '20시 발전량(MWh)': 493.92, '21시 발전량(MWh)': 590.688, '22시 발전량(MWh)': 590.4

# **3. 2022년~2024년까지의 남동 발전소 데이터 결합(훈련 데이터 생성)**

In [12]:
base_dir = Path(r"C:/ESG_Project1/file/solar_data_file")

# 기대 스키마
HOUR_COLS = [f"{i}시 발전량(MWh)" for i in range(1, 25)]
TAIL_COLS = ["총량(KW)", "평균(KW)", "최대(시간별)", "최소(시간별)", "최대", "최소"]
EXPECTED = ["발전구분", "호기", "일자"] + HOUR_COLS + TAIL_COLS

def normalize_cols(cols: pd.Index) -> pd.Index:
    """컬럼명: BOM/다중 공백 제거, '1 시 발전량' → '1시 발전량' 통일"""
    cols = (cols
            .str.replace('\ufeff', '', regex=False)
            .str.replace(r'\s+', ' ', regex=True)
            .str.strip())
    cols = cols.str.replace(r"^(\d+)\s*시\s*발전량\(MWh\)$",
                            lambda m: f"{m.group(1)}시 발전량(MWh)", regex=True)
    return cols

def read_csv_safely(path: Path) -> pd.DataFrame:
    """인코딩/구분자 자동 감지 + 헤더 정리 + 스키마 맞춤"""
    last_err = None
    for enc in ("utf-8-sig", "utf-8", "cp949", "euc-kr"):
        try:
            # 구분자 자동 감지
            df = pd.read_csv(path, encoding=enc, sep=None, engine="python")
            break
        except Exception as e:
            last_err = e
            continue
    else:
        raise RuntimeError(f"인코딩/구분자 감지 실패: {path}\n{last_err}")

    # 헤더 정리
    df.columns = normalize_cols(df.columns)

    # 우발적 인덱스 컬럼 제거
    if df.columns[0].lower() in {"unnamed: 0", "index"}:
        df = df.drop(columns=[df.columns[0]])

    # 파일 정보 보강(파일명: 남동발전량_2022_01.csv)
    df["source_file"] = path.name
    parts = path.stem.split("_")
    if len(parts) >= 3 and parts[-2].isdigit() and re.fullmatch(r"\d{2}", parts[-1]):
        df["연"] = parts[-2]
        df["월"] = parts[-1]

    # 시간열 표기 변형 자동 매핑(안전망)
    fix = {}
    for c in df.columns:
        m = re.match(r"^(\d+)\s*시\s*발전량\(MWh\)$", c)
        if m:
            fix[c] = f"{int(m.group(1))}시 발전량(MWh)"
    if fix:
        df = df.rename(columns=fix)

    # 누락 열 생성 후 기대 스키마 순서로 정렬
    for col in EXPECTED:
        if col not in df.columns:
            df[col] = pd.NA
    df = df[[c for c in EXPECTED if c in df.columns] +
            [c for c in df.columns if c not in EXPECTED]]

    return df

# === 1) 파일 목록(2022~2024) ===
files = (sorted((base_dir / "2022").glob("*.csv")) +
         sorted((base_dir / "2023").glob("*.csv")) +
         sorted((base_dir / "2024").glob("*.csv")))

# === 2) 병합 ===
all_df = pd.concat([read_csv_safely(p) for p in files], ignore_index=True, sort=False)
print("2022~2024년 데이터 통합 완료")
print("파일 개수:", len(files))
print("DataFrame 크기:", all_df.shape)
print("열 목록:", list(all_df.columns)[:10], "...")

# === 3) 밀린 행 자동 복구 ===
def is_non_numeric(x):
    try:
        float(str(x))
        return False
    except Exception:
        return True

if "발전구분" in all_df.columns and "호기" in all_df.columns:
    cols_in_use = [c for c in (["발전구분", "호기", "일자"] + HOUR_COLS + TAIL_COLS) if c in all_df.columns]
    mask = all_df["발전구분"].isna() & all_df["호기"].apply(is_non_numeric)
    if mask.any():
        tmp = all_df.loc[mask, cols_in_use].copy()
        # 오른쪽으로 1칸 이동: cols[1:] <- cols[:-1]
        all_df.loc[mask, cols_in_use[1:]] = tmp[cols_in_use[:-1]].values
        # 첫 칸(발전구분)은 원래 호기에 있던 값으로
        all_df.loc[mask, cols_in_use[0]] = tmp[cols_in_use[1]]
        print("밀림 복구 행 수:", int(mask.sum()))
    else:
        print("밀림 복구 대상 없음")

# === 4) 타입 정리 ===
if "일자" in all_df.columns:
    all_df["일자"] = pd.to_datetime(all_df["일자"], errors="coerce")

num_cols = [c for c in HOUR_COLS + TAIL_COLS if c in all_df.columns]
for c in num_cols:
    all_df[c] = pd.to_numeric(all_df[c], errors="coerce")

print(all_df.head(5))

# === 5) (선택) 저장 ===
# out_path = base_dir / "남동발전_2022_2024_통합_정리.csv"
# all_df.to_csv(out_path, index=False, encoding="utf-8-sig")
# print("저장 완료:", out_path)

2022~2024년 데이터 통합 완료
파일 개수: 36
DataFrame 크기: (24450, 36)
열 목록: ['발전구분', '호기', '일자', '1시 발전량(MWh)', '2시 발전량(MWh)', '3시 발전량(MWh)', '4시 발전량(MWh)', '5시 발전량(MWh)', '6시 발전량(MWh)', '7시 발전량(MWh)'] ...
밀림 복구 대상 없음
   발전구분                    호기         일자  1시 발전량(MWh)  2시 발전량(MWh)  \
0     1   2022-01-31 00:00:00 1970-01-01          0.0          0.0   
1     1   2022-01-31 00:00:00 1970-01-01          0.0          0.0   
2     2   2022-01-31 00:00:00 1970-01-01          0.0          0.0   
3     1   2022-01-31 00:00:00 1970-01-01          0.0          0.0   
4     1   2022-01-31 00:00:00 1970-01-01          0.0          0.0   

   3시 발전량(MWh)  4시 발전량(MWh)  5시 발전량(MWh)  6시 발전량(MWh)  7시 발전량(MWh)  ...  \
0          0.0          0.0          0.0          0.0        0.000  ...   
1          0.0          0.0          0.0          0.0        3.820  ...   
2          0.0          0.0          0.0          0.0        5.094  ...   
3          0.0          0.0          0.0          0.0        4.800  ...   

In [13]:
bad = all_df[all_df["발전구분"].isna() & all_df["호기"].apply(is_non_numeric)]
print(bad["source_file"].value_counts().head())

Series([], Name: count, dtype: int64)


# **4. 결합한 데이터 저장(훈련 데이터 생성)** 

In [ ]:
# 통합된 DataFrame을 CSV로 저장
output_path = r"C:/ESG_Project1/file/solar_data_file/train.csv"
all_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"CSV 저장 완료: {output_path}")

# **5. 2025년 남동 발전소 데이터 결합(테스트 데이터 생성)**

# **6. 결합한 데이터 저장(테스트 데이터 생성)**

저장 완료: C:/ESG_Project1/file/solar_data_file/test.csv
